In [1]:
from azure.quantum import Workspace
workspace = Workspace (
   resource_id = "/subscriptions/8f5de5cb-da38-4417-990e-457c3292608e/resourceGroups/bitsdemo-new/providers/Microsoft.Quantum/Workspaces/bitsdemonew",
   location = "eastus"
)


In [2]:
from qiskit import *
from qiskit.utils import QuantumInstance

In [2]:
# Import necesary packages
# ! pip install imbalanced-learn
# ! pip install lazypredict
# ! pip install qiskit
# ! pip install qiskit-machine-learning
import numpy as np
import pandas as pd




In [4]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider(
  resource_id="/subscriptions/8f5de5cb-da38-4417-990e-457c3292608e/resourceGroups/bitsdemo-new/providers/Microsoft.Quantum/Workspaces/bitsdemonew",
  location="eastus"
)

# Create IonQ simulator and QPU backends
ionq_simulator_backend = provider.get_backend("ionq.simulator")
ionq_qpu_backend = provider.get_backend("ionq.qpu.aria-1")

# Create Rigetti simulator and QPU backends
rigetti_simulator_backend = provider.get_backend("rigetti.sim.qvm")
rigetti_qpu_backend = provider.get_backend("rigetti.qpu.aspen-m-3")

# Create Quantinuum simulator and QPU backends
quantinuum_simulator_backend = provider.get_backend("quantinuum.sim.h1-2sc")
quantinuum_qpu_backend = provider.get_backend("quantinuum.qpu.h1-2")

# Set your backend of choice
backend = quantinuum_simulator_backend

In [5]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
from qiskit.utils import algorithm_globals

from qiskit_machine_learning.algorithms import VQC

In [3]:
# read the Cleaned up MAWS Department CSV
df = pd.read_csv('https://raw.githubusercontent.com/karthikeyanVK/MAWSPredictionModel/main/maws-cleaned.csv')

In [4]:
df.head()

,projectValue,startDate_delay,technicalSanctionDate_delay,tenderAnnounced_delay,technicalBidFinalized_delay,priceBidFinalized_delay,workOrderIssued_delay,agreementDate_delay,workCommenced_delay,works25CompletedDate_delay,...,Name_of_Commissioner_MAWS Vaniyambadi,Name_of_Commissioner_MAWS Vedaranyam,Name_of_Commissioner_MAWS Vellakovil,Name_of_Commissioner_MAWS Vikramasingarapuram,Name_of_Commissioner_MAWS Villupuram,Name_of_Commissioner_MAWS Virudhachalam,Name_of_Commissioner_MAWS Virudhunagar,Name_of_Commissioner_MAWS Walajapet,projectGrade_B,projectGrade_C
0,180000.0,0,-15.0,0.0,-12.0,-3.0,5.0,-7.0,76.0,-14.0,...,0,0,0,0,0,0,0,0,0,1
1,400000.0,0,-30.0,-3.0,-12.0,-3.0,-3.0,-7.0,-7.0,-11.0,...,0,0,0,0,0,0,0,0,0,1
2,3000000.0,0,10.0,110.0,-20.0,0.0,-3.0,-7.0,9.0,223.0,...,0,0,0,0,0,0,0,0,0,1
3,8055000.0,0,6.0,5.0,-15.0,-2.0,6.0,-7.0,24.0,24.0,...,0,0,0,0,0,0,0,0,1,0
4,300000.0,0,0.0,-3.0,-20.0,-3.0,15.0,-7.0,-6.0,-22.0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
# https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
df = df.replace(np.nan, 0,regex=True)
df.fillna(0)

from sklearn.model_selection import train_test_split
X = df.loc[:, df.columns != 'delayed']
y = df.loc[:, df.columns == 'delayed']

X.head()

y = np.nan_to_num(y).ravel() 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns


In [6]:

from qiskit.circuit.library import ZZFeatureMap
from qiskit.algorithms.optimizers import SPSA
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.algorithms import VQC


# Load dataset
feature_dim = X_train.shape[1]

# feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2)
# var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
# feature_map = feature_map.compose(var_form)


In [ ]:
# # Create a VQC instance with the SPSA optimizer and the feature map
# vqc = VQC(optimizer=SPSA(maxiter=100), feature_map=feature_map,quantum_instance=QuantumInstance(backend))



In [7]:
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement="linear")
ansatz = TwoLocal(feature_map.num_qubits, ["ry", "rz"], "cz", reps=3)
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=COBYLA(maxiter=100),
    quantum_instance=QuantumInstance(backend),
)


/tmp/ipykernel_7021/2517815466.py:3: DeprecationWarning: The quantum_instance argument is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the sampler argument.
  vqc = VQC(


In [8]:
vqc.fit(X_train, X_test)

score = vqc.score(y_train, y_test)


...............................................................

In [ ]:
print(f"Testing accuracy: {score:0.2f}")